## Load Credentials from a JSON

In [55]:
import json
import tweepy
import re
from collections import Counter
import time
from decouple import config
#!pip install trio
import trio
import sys

In [6]:
#Load Twitter Credentials File
'''TWITTER_AUTH = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))

TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))

TWITTER = tweepy.API(TWITTER_AUTH)'''


with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    TWITTER_AUTH = tweepy.OAuthHandler(info['CONSUMER_KEY'], info['CONSUMER_SECRET'])
    TWITTER_AUTH.set_access_token(info['ACCESS_KEY'],info['ACCESS_SECRET'])

TWITTER = tweepy.API(TWITTER_AUTH);


## Function 1 - First Layer Interactions

## ---------This is broken ---------------------------------------- 

In [197]:
async def first_async_get_user_interactions(search, output, next_query):
    # Initialize search for the specific user.
    # Get the user's tweets from their timeline. 
    # Cycle through all the tweets' text and join it into a mega-string.
    # Do some standardizing and replacing.
    # Strip away everything except usernames, into a string. 
    # Make a list of the counts, and take the top (X) most common people. 
    # Creates a list of the `top` people. No duplicates 
    # Tuple the results of the search & output together.
    # Make search inputs for the next level of crawling. 
    twitter_user = TWITTER.get_user(search)
    tweets = await fetch_tweets(twitter_user)
    b = " ".join(tweets)
    b = b.lower()
    b = b.replace(search, "")
    out = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    top = Counter(out).most_common(20)
    interactions = []
    for interaction_count in top:
        interactions += ([interaction_count[0]] * interaction_count[1])    
    tweet_data = [(search, i) for i in interactions]
    
    output.extend(tweet_data)
    
    next_target_users = [person[0] for person in top]
    
    next_query.extend(next_target_users)

In [284]:
async def hit_api(twitter_user, loop_num, start_time, tweets):
    print('Beginning API call in Process # :' , loop_num, " . Time so far:", time.time() - start_time)
    raw_tweets = twitter_user.timeline(
                count=200,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended')
    out_tweets = [ i.full_text for i in raw_tweets ]
    tweets.extend(out_tweets)
    print('API call has finished in Process # :' , loop_num, " . Time so far:", time.time() - start_time)

In [285]:
async def fetch_tweets(twitter_user, loop_num, start_time, tweets):
    print('Beginning API call in Process # :' , loop_num, " . Time so far:", time.time() - start_time)
    raw_tweets = twitter_user.timeline(
                count=200,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended')
    out_tweets = [ i.full_text for i in raw_tweets ]
    tweets.extend(out_tweets)

In [314]:
async def fetch_tweets_test(twitter_user, loop_num, start_time, tweets):
    print('Beginning Fetch Tweets in Process # :', loop_num, " . Time so far:", time.time() - start_time)
    await (hit_api(twitter_user, loop_num, start_time, tweets))
    print('Fetch Tweets has finished! in Process # :' , loop_num, " . Time so far:", time.time() - start_time)
    #tweets.extend(out_tweets)

## Function 2 - Second Layer Interactions

In [325]:
async def second_async_get_user_interactions(search, output, loop_num, start_time, nursery):
    # Initialize search for the specific user. 
    twitter_user = TWITTER.get_user(search)
    # Get the user's tweets from their timeline. 
    
    print('Grabbing Tweets in Process #: ', loop_num, " . Time so far:", time.time() - start_time)
    tweets = []
    await fetch_tweets_test(twitter_user, loop_num, start_time, tweets)
    
    #nursery.start_soon(fetch_tweets, twitter_user, loop_num, start_time, tweets)
    
    
    
    # Cycle through all the tweets' text and join it into a mega-string.
    print('Beginning Regex Processing in Process #: ', loop_num, " . Time so far:", time.time() - start_time)
    b = " ".join(tweets)
    # Do some standardizing and replacing.
    b = b.lower()
    b = b.replace(search, "")
    # Strip away everything except usernames, into a string. 
    out = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    print('Finished Regex Processing in Process #: ', loop_num, " . Time so far:", time.time() - start_time)
    # Tuple the results of the search & output together.
    tweet_data = [(search, i) for i in out]
    output = output.extend(tweet_data)
    print('Process# ', loop_num, "Finished... Time so far:", time.time() - start_time)

## Func 3 - Looper


In [326]:
async def looper (user, data, nursery, start_time, loop_num):
    print("Beginning Child Process # ", loop_num, "-----user is: ", user, " . Time so far:", time.time() - start_time)
    nursery.start_soon(second_async_get_user_interactions, user, data, loop_num, start_time, nursery)
    #print('')
            

## Trio Async Tests

## 1. Drill down. (No 1st Layer Grouping) 

In [327]:
async def interaction_chain(first_user):
    data = []
    next_users = ['austen', 'webdevmason', 'bwinterrose']
    start_time = time.time()
    
    # Level 1 Run. 
    #await first_async_get_user_interactions(first_user, data, next_users)

    print("----------Entering 2nd level. Time so far:", time.time() - start_time)
    display(next_users)
    
    # Level 2 Run. 
    loop_num = 1
    async with trio.open_nursery() as nursery:
           for user in next_users:
                await looper(user, data, nursery, start_time, loop_num)
                loop_num += 1

    print("Total time:", time.time() - start_time)
    return data

In [328]:
data = trio.run(interaction_chain, 'austen')

----------Entering 2nd level. Time so far: 2.384185791015625e-07


['austen', 'webdevmason', 'bwinterrose']

Beginning Child Process #  1 -----user is:  austen  . Time so far: 0.00231170654296875
Beginning Child Process #  2 -----user is:  webdevmason  . Time so far: 0.0024585723876953125
Beginning Child Process #  3 -----user is:  bwinterrose  . Time so far: 0.0025968551635742188
Grabbing Tweets in Process #:  1  . Time so far: 0.4307270050048828
Beginning Fetch Tweets in Process # : 1  . Time so far: 0.4308593273162842
Beginning API call in Process # : 1  . Time so far: 0.4311809539794922
API call has finished in Process # : 1  . Time so far: 1.5040056705474854
Fetch Tweets has finished! in Process # : 1  . Time so far: 1.5043790340423584
Beginning Regex Processing in Process #:  1  . Time so far: 1.5044167041778564
Finished Regex Processing in Process #:  1  . Time so far: 1.5053791999816895
Process#  1 Finished... Time so far: 1.5055742263793945
Grabbing Tweets in Process #:  3  . Time so far: 1.9444825649261475
Beginning Fetch Tweets in Process # : 3  . Time so far: 1.9448316097259521
Be

## To Optimize this further, we need to make it do the API calls concurrently. 

## The Above reads:

```
Grabbing Tweets in Process #:  1  . Time so far: 0.4307270050048828
Beginning Fetch Tweets in Process # : 1  . Time so far: 0.4308593273162842
Beginning API call in Process # : 1  . Time so far: 0.4311809539794922
API call has finished in Process # : 1  . Time so far: 1.5040056705474854
Fetch Tweets has finished! in Process # : 1  . Time so far: 1.5043790340423584
Beginning Regex Processing in Process #:  1  . Time so far: 1.5044167041778564
Finished Regex Processing in Process #:  1  . Time so far: 1.5053791999816895
Process#  1 Finished... Time so far: 1.5055742263793945
Grabbing Tweets in Process #:  3  . Time so far: 1.9444825649261475
```


## We need to order is so that it reads:
```
Grabbing Tweets in Process #:  1  . Time so far: 0.4307270050048828
Beginning Fetch Tweets in Process # : 1  . Time so far: 0.4308593273162842
Beginning API call in Process # : 1  . Time so far: 0.4311809539794922
Beginning API call in Process # : 3 --------------------
API call has finished in Process # : 1  . Time so far: 1.5040056705474854
Fetch Tweets has finished! in Process # : 1  . Time so far: 1.5043790340423584
API call has finished in Process # : 3 Time so far: ---------------

```


## In order to get a noticable speedup, the algorithm needs to open more API requests while it's waiting for the responses


In [329]:
data

[('austen', 'tiggreen'),
 ('austen', 'swyx'),
 ('austen', 'ryanallred'),
 ('austen', 'ryanallred'),
 ('austen', 'ryanallred'),
 ('austen', 'ryanallred'),
 ('austen', 'honestduane'),
 ('austen', 'honestduane'),
 ('austen', 'honestduane'),
 ('austen', 'honestduane'),
 ('austen', 'honestduane'),
 ('austen', 'honestduane'),
 ('austen', 'honestduane'),
 ('austen', 'honestduane'),
 ('austen', 'naksecurity'),
 ('austen', 'knox_jarvis'),
 ('austen', 'bamcphail'),
 ('austen', 'dustoftherabbi'),
 ('austen', 'mispelledtoyota'),
 ('austen', 'lambdaschool'),
 ('austen', 'erostribe'),
 ('austen', 'lambdaschool'),
 ('austen', 'erostribe'),
 ('austen', 'lambdaschool'),
 ('austen', 'techinclusionco'),
 ('austen', 'erostribe'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'justinandrade_j'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'please_be_smart'),
 ('austen', 'vladstan'),
 ('austen', 'jwdanner'),
 ('austen', 'jamiesmallwood'),
 ('austen', 'subes0